In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
df0 = pd.read_csv('data/annual_csi_data_for_match.csv', sep=',')

In [3]:
df0.rename(index=str, columns={"datatime": "datetime"}, inplace=True)

In [4]:
df0['datetime'] = pd.to_datetime(df0['datetime'])

In [5]:
df0.head()

,datetime,lat,lon,city,actualkwh,irradiancekwh,pr,mount_type,koeppen,capacity_dc_kw,tilt,azimuth,paneltype,cellcategory,cod,invertermake,invertermodel,panelmake,panelmodel
0,2009-01-01,36.817933,-119.759976,Fresno,215457.0875,278138.156430,0.774641,Fixed - Roof Mounted,Moderate,161.120,30.0,180.0,mono,Premium,2008-09-05,SatCon,AE-135-60-PV-A,SunPower,PL-EVER-ES-190P
1,2010-01-01,36.817933,-119.759976,Fresno,193487.6600,261959.131597,0.738618,Fixed - Roof Mounted,Moderate,161.120,30.0,180.0,mono,Premium,2008-09-05,SatCon,AE-135-60-PV-A,SunPower,PL-EVER-ES-190P
2,2009-01-01,34.016505,-118.113753,Montebello,288246.6500,416118.658276,0.692703,Fixed - Roof Mounted,Moderate,243.200,30.0,180.0,mono,Premium,2007-08-01,Xantrex,PV225S-480-P,SunPower,PL-EVER-ES-190P
3,2010-01-01,34.016505,-118.113753,Montebello,252797.2250,409327.120794,0.617592,Fixed - Roof Mounted,Moderate,243.200,30.0,180.0,mono,Premium,2007-08-01,Xantrex,PV225S-480-P,SunPower,PL-EVER-ES-190P
4,2009-01-01,33.782519,-117.228648,Perris,56125.0000,81674.965560,0.687175,Fixed - Roof Mounted,Moderate,46.592,30.0,180.0,poly,Standard,2007-10-15,SatCon,AE-50-60-PV-D,Sharp,ND-208U2


In [6]:
df0.reset_index(drop = True, inplace = True)

In [7]:
df0.shape

(8157, 19)

In [51]:
# Function to inspect all value counts at once excluding 'lat' & 'lon'
def val_counts(df):
    for i in df.columns:
        if i not in ['lat', 'lon']:
            print(df[f'{i}'].value_counts())

In [56]:
def dt_to_year(df):
    df['year'] = np.zeros(df.shape[0])
    for i, el in df['datetime'].iteritems():
        df.at[i,'year'] = el.year
    df['year'] = df['year'].astype(int)
    df.drop(['datetime'], axis=1, inplace=True)

In [57]:
def pop_year(yr):
    mask = df['year']==yr
    df_new = df[mask]
    return df_new

In [61]:
df = df0.copy()

In [62]:
dt_to_year(df)

In [63]:
df08 = pop_year(2008)

In [64]:
def clean_data(df):    
    drop_list = ['city', 'cod', 'panelmodel','invertermodel','lon','pr','invertermake','panelmake']
    df.drop(drop_list, axis=1, inplace=True)
    df.paneltype.fillna(value = 'poly', inplace=True)
    df.mount_type.fillna(value = 'Fixed - Roof Mounted', inplace=True)
    df['cellcategory'].replace('Unknown', 'Standard', inplace=True)
    df['azimuth'].replace('Mixed', 180, inplace=True)
    df['azimuth'].astype(float)
    # df['tilt_mixed'] = (df['tilt']=='Mixed')*1
    df['tilt'].replace('Mixed', round(df['lat'],1),inplace=True)
    df['tilt'].astype(float)
    return df

In [65]:
clean_data(df08)

/home/matchdt/miniconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/matchdt/miniconda3/lib/python3.6/site-packages/pandas/core/generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/matchdt/miniconda3/lib/python3.6/site-packages/pandas/core/generic.py:5886: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,lat,actualkwh,irradiancekwh,mount_type,koeppen,capacity_dc_kw,tilt,azimuth,paneltype,cellcategory,year
58,33.751342,4.542028e+05,6.160226e+05,Fixed - Roof Mounted,Moderate,348.480,30.0,180.0,mono,Premium,2008
153,38.440429,2.744079e+05,3.029105e+05,Fixed - Roof Mounted,Moderate,198.720,20.0,180.0,poly,Standard,2008
166,36.838139,6.266282e+05,6.578710e+05,Fixed - Roof Mounted,Moderate,406.080,20.0,180.0,poly,Standard,2008
174,37.507159,4.533352e+03,5.121172e+03,Fixed - Roof Mounted,Moderate,3.150,18.0,200.0,poly,Standard,2008
187,37.983533,1.058762e+05,1.123506e+05,Fixed - Roof Mounted,Moderate,70.980,10.0,170.0,mono,Premium,2008
192,38.678516,1.414849e+06,1.992794e+06,Fixed - Roof Mounted,Moderate,1182.070,15.0,180.0,poly,Mixed,2008
204,38.356577,4.715546e+05,5.279643e+05,Fixed - Roof Mounted,Moderate,348.480,20.0,180.0,poly,Standard,2008
238,37.386812,2.703383e+03,3.860747e+03,Fixed - Roof Mounted,Moderate,2.460,20.0,215.0,mono,Premium,2008
249,37.941034,1.138300e+04,1.164903e+04,Fixed - Roof Mounted,Moderate,7.000,21.0,180.0,poly,Standard,2008
273,39.150171,2.371500e+04,2.345281e+04,Fixed - Roof Mounted,Moderate,15.510,12.0,180.0,poly,Standard,2008


In [212]:
#val_counts(df)

In [214]:
# df.isna().sum()

In [215]:
#df.dropna(axis=0, inplace=True)

In [26]:
# df[df.paneltype.isna()]


In [238]:
# df['tilt_mixed'] = np.zeros(df.shape[0])
# for i, el in df['tilt'].iteritems():
#     if el == 'mixed':
#         df.at[i,'tilt'] = 0
#         df.at[i,'tilt_mixed'] = 1
        

In [27]:
df['tilt_mixed'] = (df['tilt']=='Mixed')*1

In [28]:
df['tilt_mixed'].value_counts()

0    7286
1     871
Name: tilt_mixed, dtype: int64

In [222]:
df['tilt'].value_counts()

20.0     1089
10.0      909
Mixed     871
5.0       832
18.0      589
15.0      586
30.0      491
0.0       389
22.0      318
17.0      244
25.0      167
23.0      158
19.0      150
16.0      137
21.0      132
14.0      126
24.0      112
12.0      108
26.0       96
7.0        83
13.0       57
8.0        53
27.0       52
11.0       46
9.0        39
28.0       39
33.0       33
2.0        32
34.0       29
3.0        28
6.0        25
4.0        25
35.0       23
31.0       17
40.0       16
32.0       16
1.0        11
45.0        8
29.0        7
37.0        5
36.0        2
42.0        2
50.0        2
72.0        1
38.0        1
54.0        1
Name: tilt, dtype: int64

In [29]:
df['rd_lat'] = round(df['lat'],1)
df['rd_lat']

0       36.8
1       36.8
2       34.0
3       34.0
4       33.8
5       33.8
6       38.0
7       38.0
8       34.0
9       33.6
10      33.6
11      37.4
12      37.5
13      37.5
14      39.6
15      33.9
16      33.9
17      37.4
18      37.4
19      33.7
20      33.7
21      34.3
22      34.3
23      37.4
24      37.4
25      37.4
26      38.0
27      38.0
28      38.5
29      33.9
        ... 
8127    33.2
8128    33.0
8129    33.4
8130    33.2
8131    33.2
8132    33.0
8133    32.8
8134    33.2
8135    32.8
8136    32.8
8137    32.8
8138    33.1
8139    33.1
8140    33.1
8141    32.8
8142    32.8
8143    32.6
8144    33.0
8145    32.9
8146    33.2
8147    33.2
8148    32.7
8149    32.8
8150    32.8
8151    33.0
8152    33.5
8153    33.1
8154    33.1
8155    33.4
8156    33.1
Name: rd_lat, Length: 8157, dtype: float64

In [43]:
df['tilt'].replace('Mixed', round(df['lat'],1),inplace=True)

In [44]:
df['tilt'].astype(float)

0       30.0
1       30.0
2       30.0
3       30.0
4       30.0
5       30.0
6       30.0
7       30.0
8       20.0
9        0.0
10       0.0
11      30.0
12      30.0
13      30.0
14      30.0
15      30.0
16      30.0
17      30.0
18      30.0
19      30.0
20      30.0
21      30.0
22      30.0
23      30.0
24      30.0
25      30.0
26      30.0
27      30.0
28      30.0
29      30.0
        ... 
8127    23.0
8128    18.0
8129    18.0
8130     8.0
8131     8.0
8132    18.0
8133    15.0
8134    30.0
8135    18.0
8136    18.0
8137    20.0
8138    15.0
8139    15.0
8140    22.0
8141    18.0
8142    32.8
8143    26.0
8144    15.0
8145    20.0
8146    10.0
8147    10.0
8148    30.0
8149    15.0
8150    15.0
8151    14.0
8152    33.5
8153     5.0
8154     5.0
8155    18.0
8156    20.0
Name: tilt, Length: 8157, dtype: float64

In [32]:
df['tilt'].value_counts()

20.0    1089
10.0     909
5.0      832
18.0     589
15.0     586
30.0     491
0.0      389
22.0     318
17.0     244
25.0     167
23.0     158
19.0     150
16.0     137
21.0     132
14.0     126
24.0     112
12.0     108
26.0      96
7.0       83
13.0      57
8.0       53
27.0      52
37.9      48
11.0      46
37.3      43
34.6      42
33.7      40
28.0      39
9.0       39
38.5      35
        ... 
37.0       5
37.0       5
32.7       5
39.3       4
39.4       4
36.4       4
36.6       4
39.7       4
35.0       4
35.3       3
36.7       3
39.2       3
32.9       3
39.5       3
50.0       2
40.6       2
37.1       2
36.0       2
36.3       2
35.1       2
39.9       2
42.0       2
38.0       1
54.0       1
33.3       1
33.2       1
72.0       1
35.5       1
33.4       1
39.0       1
Name: tilt, Length: 107, dtype: int64

In [139]:
# val_counts(df)

In [140]:
# df.isna().sum()

In [141]:
####  df.sort_values(by =['pr'], ascending=False)

In [25]:
df_08 = pop_year(2008)